In [1]:
from rlrom.testers import RLTester
import rlrom.utils as utils
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
from pprint import pprint
from gymnasium import spaces

Loading BokehJS ...

In [ ]:
p = figure(width=400, height=400)

p.circle([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=15, line_color="navy", 
fill_color="orange", fill_alpha=0.5)

show(p)

AttributeError: unexpected attribute 'plot_width' to figure, similar attributes are outer_width, width or min_width

In [ ]:
cfg = utils.load_cfg('cfg0_hug_specs.yml')
#cfg = utils.load_cfg('cfg_min.yml')
pprint(cfg)


In [ ]:
T = RLTester(cfg)
Tres= T.run_cfg_test()

In [ ]:
Tr = T.test_results
print(Tr)

In [ ]:
lay = """
_ep(1)
 x
 sat(far_left)
 reward
"""
fig, _=  T.get_fig(lay)
show(fig)

In [ ]:
T.env.env.get_values_from_str('x')